In [11]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import WordNetLemmatizer 
from tqdm import tqdm

In [12]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
nameOfTag = newsgroups_train.target_names

In [22]:
n_features = 4700
n_components = 20
n_top_words = 10


vectorizer = CountVectorizer(
                    lowercase=True, stop_words=_stop_words.ENGLISH_STOP_WORDS,
                    analyzer='word', binary=True,
                    max_df=0.75, min_df=2,
                    max_features=n_features
)
X_train = vectorizer.fit_transform(newsgroups_train.data).toarray()

In [27]:
def _customLDA(ndk, nkw, nk, _z, _document, _word, _alpha, _beta, _topic,  max_iter=30):
    for i in tqdm(range(max_iter)):
        for j in range(len(_document)):
            cur_word = _word[j]
            cur_document = _document[j]
            cur_topic = _z[j]
            ndk[cur_document, cur_topic] -= 1
            nkw[cur_topic, cur_word] -= 1
            nk[cur_topic] -= 1
            p = (ndk[cur_document, :] + _alpha) * (nkw[:, cur_word] + _beta[cur_word]) / (nk + _beta.sum())
            _z[j] = np.random.choice(np.arange(_topic), p = p / p.sum())
            ndk[cur_document, _z[j]] += 1
            nkw[_z[j], cur_word] += 1
            nk[_z[j]] += 1
    return ndk, nkw, nk, _z

In [24]:
topic = 20
ndk = np.zeros( topic * X_train.shape[0]).reshape(X_train.shape[0], topic)
nkw = np.zeros( topic * X_train.shape[1]).reshape(topic, X_train.shape[1])
nk = np.zeros(topic)
document, word = X_train.nonzero()
z = np.random.choice(topic, len(document))

In [26]:
for i, j, k in zip(document, word, z):
    ndk[i, k] += 1
    nkw[k, j] += 1
    nk[k] += 1

In [28]:
ndk, nkw,  nk, z = _customLDA(ndk, nkw, nk, z, document, word, np.ones(20), np.ones(X_train.shape[1]), 20, max_iter=30)

100%|██████████| 30/30 [12:52<00:00, 25.75s/it]


при max_df =0.95 получал слишком много общеупотребительных слов, уменьшил до 0.75 + увеличил количество итераций

In [29]:
result = np.argsort(nkw, axis=1)[:, -10:]
for i in range(20):
    matrix = np.zeros((1, X_train.shape[1]))
    for j in result[i]:
        matrix[0, j] = 1
    print('Tag {} \t{}'.format(i + 1, '\t'.join(vectorizer.inverse_transform(matrix)[0])))

Tag 1 	earth	edu	gordon	nasa	research	science	soon	space	time	university
Tag 2 	current	known	line	list	new	order	posted	posting	read	way
Tag 3 	believe	bible	christian	does	god	jesus	life	people	say	true
Tag 4 	don	isn	just	know	like	people	things	think	time	ve
Tag 5 	ago	did	just	like	people	right	seen	time	ve	years
Tag 6 	file	files	ftp	program	software	use	using	version	window	windows
Tag 7 	case	don	government	gun	law	make	people	right	rights	state
Tag 8 	address	discussion	example	general	group	groups	having	number	point	questions
Tag 9 	chip	clipper	data	encryption	government	key	keys	phone	public	use
Tag 10 	card	computer	disk	drive	hard	mac	memory	monitor	use	video
Tag 11 	10	11	12	13	14	15	16	20	25	30
Tag 12 	day	did	going	got	left	little	right	started	took	went
Tag 13 	cost	make	money	new	offer	pay	price	sale	sell	used
Tag 14 	government	history	israel	jews	killed	people	state	war	world	years
Tag 15 	actually	called	com	does	good	read	try	use	want	way
Tag 16 	doesn	don	just	

можно выделить несколько тегов которые выделяются в 1 тему:
1 космос
2 интернет
3 вера
6 программирование
7 обсуждение возможности владеть оружием?
И ТД